In [1]:
GCOLLAB = False
GIT_FOLDER = './'

if GCOLLAB :
  from google.colab import drive
  drive.mount('/content/drive/')
  GIT_FOLDER = '/content/drive/MyDrive/4_Open_classrooms/OC_DS_P7/'

  import importlib.util
  import sys
  spec = importlib.util.spec_from_file_location("custom_kernel", GIT_FOLDER+"custom_kernel.py")
  ckk = importlib.util.module_from_spec(spec)
  sys.modules["custom_kernel"] = ckk
  spec.loader.exec_module(ckk)

else :
  import mlflow
  import custom_kernel as ckk

In [2]:
import pandas as pd
import numpy as np
import gc

In [3]:
import time
from contextlib import contextmanager

In [4]:
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(name, time.time() - t0))

In [5]:
INPUT_FOLDER = GIT_FOLDER + 'input_data/'

In [6]:
# ogdf = pd.read_csv(INPUT_FOLDER+'application_train.csv')
# df = ckk.application_train_test(ds_folder=INPUT_FOLDER)
# print("Forme de l'app_train original : ", ogdf.shape)
# print("Forme de l'app_train transforme : ", df.shape)
# print()
#
# print("---")
# ogbureau = pd.read_csv(INPUT_FOLDER+'bureau.csv')
# print("Forme de bureau original : ", ogbureau.shape)
# ogbureaubalance = pd.read_csv(INPUT_FOLDER+'bureau_balance.csv')
# dfb = ckk.process_bureau(path=INPUT_FOLDER)
# df = pd.merge(df, dfb, on='SK_ID_CURR', how='left')
# print("Forme de bureau balance original : ", ogbureaubalance.shape)
# print("Forme du bureau transforme : ", dfb.shape)
# print()
#
# print("---")
# ogprevious = pd.read_csv(INPUT_FOLDER+'previous_application.csv')
# print("Forme de PreviousApp original : ", ogprevious.shape)
# dfpr = ckk.get_previous_applications(path=INPUT_FOLDER)
# df = pd.merge(df, dfpr, on='SK_ID_CURR', how='left')
# print("Forme du PreviousApp transforme : ", dfpr.shape)
# print()
#
# print("---")
# ogposcash = pd.read_csv(INPUT_FOLDER+'POS_CASH_balance.csv')
# print("Forme de poscash original : ", ogposcash.shape)
# dfposcash = ckk.get_pos_cash(path=INPUT_FOLDER)
# df = pd.merge(df, dfposcash, on='SK_ID_CURR', how='left')
# print("Forme du poscash transforme : ", dfposcash.shape)
# print()
#
# print("---")
# install_df_og = pd.read_csv(INPUT_FOLDER+'installments_payments.csv')
# print("Forme de installment original : ", install_df_og.shape)
# install_df = ckk.kk02_get_installment_payments(path=INPUT_FOLDER)
# df = pd.merge(df, install_df, on='SK_ID_CURR', how='left')
# print("Forme du installement transforme : ", install_df.shape)
# print()
#
# del dfb, ogbureau, ogbureaubalance, dfpr, ogprevious, dfposcash, ogposcash, install_df_og, install_df; gc.collect()

In [7]:
with timer("application_train and application_test"):
    df = ckk.application_train_test(ds_folder=INPUT_FOLDER)
    print("Application dataframe shape: ", df.shape)
with timer("Bureau and bureau_balance data"):
    bureau_df = ckk.process_bureau(path=INPUT_FOLDER)
    df = pd.merge(df, bureau_df, on='SK_ID_CURR', how='left')
    print("Bureau dataframe shape: ", bureau_df.shape)
    del bureau_df; gc.collect()
with timer("previous_application"):
    prev_df = ckk.get_previous_applications(path=INPUT_FOLDER)
    df = pd.merge(df, prev_df, on='SK_ID_CURR', how='left')
    print("Previous dataframe shape: ", prev_df.shape)
    del prev_df; gc.collect()
with timer("previous applications balances"):
    pos = ckk.get_pos_cash(path=INPUT_FOLDER)
    df = pd.merge(df, pos, on='SK_ID_CURR', how='left')
    print("Pos-cash dataframe shape: ", pos.shape)
    del pos; gc.collect()
    ins = ckk.kk03_installments_payments(path=INPUT_FOLDER) # kk03 beaucoup plus rapide mais moins bon
    df = pd.merge(df, ins, on='SK_ID_CURR', how='left')
    print("Installments dataframe shape: ", ins.shape)
    del ins; gc.collect()
    cc = ckk.get_credit_card(path=INPUT_FOLDER)
    df = pd.merge(df, cc, on='SK_ID_CURR', how='left')
    print("Credit card dataframe shape: ", cc.shape)
    del cc; gc.collect()
# Add ratios and groupby between different tables
df = ckk.add_ratios_features(df)
df = ckk.reduce_memory(df)

Train samples: 307511, test samples: 48744
Application dataframe shape:  (356250, 120)
application_train and application_test - done in 5s
Bureau dataframe shape:  (305811, 76)
Bureau and bureau_balance data - done in 14s


/home/maximorose/git_repositories/oc-ds-p07/custom_kernel.py:278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  approved.loc[:, 'DAYS_LAST_DUE_DIFF'] = approved['DAYS_LAST_DUE_1ST_VERSION'] - approved['DAYS_LAST_DUE']


Previous dataframe shape:  (338857, 217)
previous_application - done in 28s
Pos-cash dataframe shape:  (337252, 21)
Installments dataframe shape:  (339587, 36)
Credit card dataframe shape:  (103558, 59)
previous applications balances - done in 99s
Initial df memory usage is 1456.83 MB for 535 columns
Final memory usage is: 478.36 MB - decreased by 67.2%


Si on supprime les colonnes avec plus de 30% de valeurs NaN, on ne trouve pas de True Positive. (cf. function "ckk.remove_too_nany_columns")

In [8]:
# ser_tg = df['TARGET']
# df_cleaned = ckk.remove_too_nany_columns(df)
# df_cleaned['TARGET'] = ser_tg
df_cleaned = df # test avec risque d'overfit

In [9]:
train_df = df_cleaned[df_cleaned['TARGET'].notnull()]
test_df = df_cleaned[df_cleaned['TARGET'].isnull()]
print("Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
del df_cleaned
gc.collect()

Train shape: (307506, 535), test shape: (48744, 535)


0

In [10]:
print("Forme train_df : ", train_df.shape)
print("nb target Nan : ", train_df['TARGET'].isna().sum())

Forme train_df :  (307506, 535)
nb target Nan :  0


In [36]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import recall_score, make_scorer
from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline # Have to use imblearn Pipeline for SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

# for SMOTE
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [12]:
X = train_df.drop(columns = ['TARGET'])
y = train_df['TARGET']

In [13]:
s_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
ss_scaler = StandardScaler()
mm_scaler = MinMaxScaler(feature_range=(0,1))
# oversample = SMOTE(k_neighbors=3, sampling_strategy=1.0)

In [14]:
test_size = 0.3
random_state = 42
# Train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
print(f"X_train : {X_train.shape} // X_test : {X_test.shape}")
print(f"y_train : {y_train.shape} // y_test : {y_test.shape}")
# estimator dummy
estimator =DummyClassifier(strategy='most_frequent')
estimator.fit(X_train, y_train)
y_pred = estimator.predict(X_test)
print("Accuracy de l'estimateur : ", estimator.score(X_test, y_test))
print("Score de rappel : ", recall_score(y_test, y_pred))

X_train : (215254, 534) // X_test : (92252, 534)
y_train : (215254,) // y_test : (92252,)
Accuracy de l'estimateur :  0.9192754628625938
Score de rappel :  0.0


On va utiliser un derive du F2 score tel que defini [ici](https://towardsdatascience.com/is-f1-the-appropriate-criterion-to-use-what-about-f2-f3-f-beta-4bd8ef17e285) :
F2 = TP/(TP + 0.2FP + 0.8FN)

In [15]:
# ---------------------------------------------------------------- MAKE SCRORER
def custom_metric(y_true, y_pred) :
    faux_negs = np.sum((y_true == 1) & (y_pred == 0))
    faux_pos = np.sum((y_true == 0) & (y_pred == 1))
    true_pos = np.sum((y_true == 1) & (y_pred == 1))
    # true_negs = np.sum((y_true == 0) & (y_pred == 0))
    f2_score = true_pos/(true_pos + 0.2 * faux_pos + 0.8 * faux_negs)
    return f2_score

custom_score = make_scorer(custom_metric, greater_is_better=True)

Best versions so far :
- estimator = RandomForestClassifier(n_estimators =  100, max_depth=15, class_weight='balanced') : Probleme est qu'il overfit de ouf
- estimator = LogisticRegression(penalty='l2', solver='lbfgs', class_weight='balanced') : A mes yeux c'est le mieux mais 30% des mauvais payeurs peuvent ne pas etre identifie comme tel.
- estimator = SGDClassifier(loss='perceptron', alpha=0.00001) : nul

In [28]:
over = SMOTE(sampling_strategy=0.1, random_state=random_state)
under = RandomUnderSampler(sampling_strategy=0.5, random_state=random_state) # {'over__sampling_strategy': 0.1, 'under__sampling_strategy': 0.5} voir GridSearch_LogReg_Over-UnderSampling

# mlflow.log_params({"DS" : "Only AppTrain, keep NaN cols"})
mlflow.autolog() # Toggle the autolog to activate/deactivate MLFLow

# estimator = RandomForestClassifier(n_estimators =  100, max_depth=15, class_weight='balanced') #
# estimator = LogisticRegression(penalty='l2', solver='lbfgs') # voir {gridtest_LogReg_CustomMetric + GridSearch_LogReg_OverUnderSampling_ClassWeight} dans MLFlow
# estimator = SGDClassifier(loss='perceptron', alpha=0.00001) # voir GridSearch_SGDClass_SMOTE_1 dans MLFlow
estimator = SVC()

# # Parametres LogisticRegression
# params = {
#     'estimator__penalty' : ['l2'],
#     'estimator__solver' : ['lbfgs', 'newton-cholesky']
# }
params = {
    'estimator__class_weight' : [None, 'balanced']
}

# # Parametres SGD
# params = {
#     'estimator__loss' : ['squared_hinge', 'perceptron'],
#     'estimator__alpha' : [0.0001, 0.001, 0.01]
# }

# # Parametres Random Forest
# params = {
#     'estimator__n_estimators' : [100,300,534],
#     'estimator__criterion' : ['gini', 'entropy', 'log_loss']
# }
params = {
    'estimator__n_estimators' : [500]
}


# Parametres SMOTE
# params = {
#     'smote__k_neighbors' : [3],
#     'smote__sampling_strategy' : [0.75, 0.8, 0.9, 1.0]
# }

# Parametres SMOTE
# params = {
#     'over__sampling_strategy' : [0.1, 0.3, 0.5],
#     'under__sampling_strategy' : [0.5, 0.4, 0.3]
# }
# params = {
#     'over__sampling_strategy' : [0.1, 0.3]
# }

pipeline = Pipeline(
    [
        ("imputer", s_imputer),
        ("scaler", mm_scaler),
        ("over", over), # SMOTE does not accept missing values encoded as NaN natively.
        ("under", under), # SMOTE does not accept missing values encoded as NaN natively.
        ("estimator", estimator)
    ]
)

cv = StratifiedKFold(n_splits=5)

model = GridSearchCV(
    estimator=pipeline,
    param_grid=params,
    cv=cv,
    scoring=custom_score,
    return_train_score=True
)

model.fit(X_train, y_train)

model

2023/05/24 18:02:25 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2023/05/24 18:02:25 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '1e0d456d5acb4ac4890aed12009a003a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2023/05/24 18:23:22 INFO mlflow.sklearn.utils: Logging the 5 best runs, no runs will be omitted.


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                       ('scaler', MinMaxScaler()),
                                       ('over',
                                        SMOTE(random_state=42,
                                              sampling_strategy=0.1)),
                                       ('under',
                                        RandomUnderSampler(random_state=42,
                                                           sampling_strategy=0.5)),
                                       ('estimator',
                                        RandomForestClassifier(class_weight='balanced',
                                                               max_depth=15))]),
             param_grid={'estimator__n_estimators': [500]},
             return_train_score=True, scoring=make_scorer(custom_metric))

In [18]:
model.best_params_

{'estimator__class_weight': 'balanced'}

In [19]:
model_top = Pipeline(
    [
        ("imputer", s_imputer),
        ("scaler", mm_scaler),
        ("over", over), # SMOTE does not accept missing values encoded as NaN natively.
        ("under", under), # SMOTE does not accept missing values encoded as NaN natively.
        # ("estimator", LogisticRegression(penalty='l2', solver='lbfgs', class_weight='balanced'))
        ("estimator", LogisticRegression(penalty='l2', solver='lbfgs', class_weight='balanced'))
    ]
)
model_top.fit(X_train, y_train)

2023/05/24 17:39:08 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a37adc05879a433d921df1c13abe9c57', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2023/05/24 17:39:08 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2023/05/24 17:39:08 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not specify a `predict` function, which is required in order to infer the signature
2023/05/24 17:39:08 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023/05/24 17:39:13 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '9bc15a1c8df942a68bb99797f0be3420', which will track hyperparameters, performance metrics, model artifacts, 

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', MinMaxScaler()),
                ('over', SMOTE(random_state=42, sampling_strategy=0.1)),
                ('under',
                 RandomUnderSampler(random_state=42, sampling_strategy=0.5)),
                ('estimator', LogisticRegression(class_weight='balanced'))])

In [33]:
y_pred = model_top.predict(X_test)

In [23]:
Xt_nonan = s_imputer.fit_transform(X_train) # Pas de NaN pour oversampler
Xt_over, yt_over = over.fit_resample(Xt_nonan, y_train) # over : sample rate : 0.1
Xt_over_under, yt_over_under = under.fit_resample(Xt_over, yt_over) # under : sample rate : 0.5
print("X_train : ", X_train.shape)
print("Over X_train : ", Xt_over.shape)
print("Over-Under X_train : ", Xt_over_under.shape)
print("--------------------")
print(f"Nombre PosClass : {yt_over_under.sum()}, soit {np.round((yt_over_under.sum()/yt_over_under.shape[0])*100, 2)}%")

2023/05/24 17:45:01 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0c591990b8434a0ab16659d78bea0c66', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2023/05/24 17:45:01 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2023/05/24 17:45:01 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not specify a `predict` function, which is required in order to infer the signature
2023/05/24 17:45:01 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


X_train :  (215254, 534)
Over X_train :  (217664, 534)
Over-Under X_train :  (59361, 534)
--------------------
Nombre PosClass : 19776.0, soit 33.31%


In [29]:
def grid_resultification(grid) :
    res = pd.DataFrame(grid.cv_results_)
    keepcols = [col for col in res.columns if "split" not in col]
    res = res[keepcols]
    # res = res.sort_values(by = ['rank_test_r2', 'rank_test_neg_mean_absolute_error'])
    return res

df_results_grid = grid_resultification(model)
df_results_grid

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator__n_estimators,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
0,157.98914,2.099721,4.851042,0.308805,500,{'estimator__n_estimators': 500},0.375468,0.007449,1,0.756201,0.002228


In [30]:
y_pred = model.best_estimator_.predict(X_test)

In [34]:
recall_score(y_test, y_pred)

0.6962535249093594

In [35]:
from sklearn.metrics import confusion_matrix

mat = np.round(confusion_matrix(y_true=y_test,y_pred=y_pred, normalize='true'), 2)
print(mat)

[[0.71 0.29]
 [0.3  0.7 ]]


In [ ]:
y_pred_proba = model_top.predict_proba(X_test)
df_rep_proba = pd.DataFrame(y_pred_proba, columns=['proba_0', 'proba_1'])
print("Forme DF proba : ", df_rep_proba.shape)
df_rep_proba['y_pred'] = y_pred
df_rep_proba['y_pred'].value_counts()

In [ ]:
df_rep_proba['y_test'] = np.array(y_test).reshape((len(y_test), 1))
df_rep_proba['y_test'].value_counts()

In [ ]:
false_negs = df_rep_proba[(df_rep_proba['y_test'] == 1) & (df_rep_proba['y_pred'] == 0)]
print("Forme des false negatives : ", false_negs.shape)
false_pos = df_rep_proba[(df_rep_proba['y_pred'] == 1) & (df_rep_proba['y_test'] == 0)]
print("Forme des false positives : ", false_pos.shape)

In [ ]:
false_negs['proba_0'].describe()

75% des faux negatifs ont une proba d'etre negatifs a plus de 77%. J'aurais espere voir des probabilite plus nuancees. Ceci est probablement du a des features dont la ponderation influe trop sur la labelisation.